## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[rep_mobile_citizens_id_city_partner]]

[[sessions_st_mobile]]

[[subscriptions_st_mobile]]


### Table

In [9]:
query_text = """--sql
CREATE TABLE db1.total_active_users_per_day_full_table
(
    `report_date` Date,
    `partner_uuid` String,
    `city` String,
    `citizen_id` Int32,
    `monetization` UInt8,
    `subscriptions_state` String,
    `address_uuid` String,
    `flat_uuid` String,
    `created_at` String,
    `activated_at` DateTime
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [10]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.total_active_users_per_day_full_table_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR 10 MINUTE TO db1.total_active_users_per_day_full_table AS
       SELECT
        r_m_c.partner_uuid AS partner_uuid,
        ses_st_m.citizen_id AS citizen_id,
        r_m_c.report_date AS report_date,
        r_m_c.monetization AS monetization,
        sub_st_m.state AS subscriptions_state,
        r_m_c.city AS city,
        r_m_c.address_uuid AS address_uuid,
        r_m_c.flat_uuid AS flat_uuid,
        c_d_m.created_at AS created_at,
        r_m_c.activated_at AS activated_at
    FROM db1.rep_mobile_citizens_id_city_partner as r_m_c
    JOIN db1.`sessions_st_mobile_ch` AS ses_st_m
        ON r_m_c.citizen_id = ses_st_m.citizen_id
        AND r_m_c.report_date = ses_st_m.report_date 
    LEFT JOIN db1.`subscriptions_st_mobile_ch` AS sub_st_m
        ON ses_st_m.citizen_id = sub_st_m.citizen_id
        AND ses_st_m.report_date = sub_st_m.report_date
    LEFT JOIN db1.`citizens_dir_mobile_ch` AS c_d_m ON c_d_m.`citizen_id`  = r_m_c.`citizen_id`
    WHERE 
        toDate(`last_use`) 
        BETWEEN toStartOfMonth(r_m_c.report_date) and r_m_c.report_date 
        AND  r_m_c.state = 'activated'
    limit 100
    --
    SETTINGS join_any_take_last_row = 1,
            join_algorithm = 'partial_merge'
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [4]:
query_text = """--sql
    SELECT
        r_m_c.partner_uuid AS partner_uuid,
        ses_st_m.citizen_id AS citizen_id,
        r_m_c.report_date AS report_date,
        r_m_c.monetization AS monetization,
        sub_st_m.state AS subscriptions_state,
        r_m_c.city AS city,
        r_m_c.address_uuid AS address_uuid,
        r_m_c.flat_uuid AS flat_uuid,
        c_d_m.created_at AS created_at,
        r_m_c.activated_at AS activated_at
    FROM db1.rep_mobile_citizens_id_city_partner as r_m_c
    JOIN db1.`sessions_st_mobile_ch` AS ses_st_m
        ON r_m_c.citizen_id = ses_st_m.citizen_id
        AND r_m_c.report_date = ses_st_m.report_date 
    LEFT JOIN db1.`subscriptions_st_mobile_ch` AS sub_st_m
        ON ses_st_m.citizen_id = sub_st_m.citizen_id
        AND ses_st_m.report_date = sub_st_m.report_date
    LEFT JOIN db1.`citizens_dir_mobile_ch` AS c_d_m ON c_d_m.`citizen_id`  = r_m_c.`citizen_id`
    WHERE 
        toDate(`last_use`) 
        BETWEEN toStartOfMonth(r_m_c.report_date) and r_m_c.report_date 
        AND  r_m_c.state = 'activated'
    limit 100
    --
    SETTINGS join_any_take_last_row = 1,
            join_algorithm = 'partial_merge'
    
    """

ch.query_run(query_text)

partner_uuid,citizen_id,report_date,monetization,subscriptions_state,city,address_uuid,flat_uuid,created_at,activated_at
str,i64,date,i64,str,str,str,str,str,datetime[ns]
"""""",601930,2023-07-13,0,"""""","""Актобе""","""74217307-0dc2-4328-a70d-a8ac8b…","""7d0d45a0-1175-42ab-a648-0bcb01…","""2022-04-07 17:58:24""",2025-01-17 13:24:43
"""""",601930,2023-07-16,0,"""""","""Актобе""","""74217307-0dc2-4328-a70d-a8ac8b…","""7d0d45a0-1175-42ab-a648-0bcb01…","""2022-04-07 17:58:24""",2025-01-17 13:24:43
"""""",601930,2023-07-21,0,"""""","""Актобе""","""74217307-0dc2-4328-a70d-a8ac8b…","""7d0d45a0-1175-42ab-a648-0bcb01…","""2022-04-07 17:58:24""",2025-01-17 13:24:43
"""""",601930,2023-07-25,0,"""""","""Актобе""","""74217307-0dc2-4328-a70d-a8ac8b…","""7d0d45a0-1175-42ab-a648-0bcb01…","""2022-04-07 17:58:24""",2025-01-17 13:24:43
"""""",601930,2023-07-26,0,"""""","""Актобе""","""74217307-0dc2-4328-a70d-a8ac8b…","""7d0d45a0-1175-42ab-a648-0bcb01…","""2022-04-07 17:58:24""",2025-01-17 13:24:43
…,…,…,…,…,…,…,…,…,…
"""0cf27bad-f4c8-4cf9-890f-bb66da…",601935,2023-08-03,1,"""""","""Саратов""","""4aed6d45-bf51-4b44-8b10-397c83…","""3cdc622a-6992-47da-a22f-4eb88e…","""2022-04-07 17:59:41""",2022-04-08 08:17:12
"""0cf27bad-f4c8-4cf9-890f-bb66da…",601935,2023-08-20,1,"""""","""Саратов""","""4aed6d45-bf51-4b44-8b10-397c83…","""3cdc622a-6992-47da-a22f-4eb88e…","""2022-04-07 17:59:41""",2022-04-08 08:17:12
"""0cf27bad-f4c8-4cf9-890f-bb66da…",601935,2023-08-24,1,"""""","""Саратов""","""4aed6d45-bf51-4b44-8b10-397c83…","""3cdc622a-6992-47da-a22f-4eb88e…","""2022-04-07 17:59:41""",2022-04-08 08:17:12


In [11]:
query_text = """--sql
    SELECT
        *
    FROM db1.total_active_users_per_day_full_table
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,partner_uuid,city,citizen_id,monetization,subscriptions_state,address_uuid,flat_uuid,created_at,activated_at
date,str,str,i64,i64,str,str,str,str,datetime[ns]
2024-08-27,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",7,0,"""deactivated""","""e697b430-cbc9-41e3-b0d0-dc02b5…","""d0c82be0-85c3-410a-ae25-413e52…","""2020-12-19 15:03:47""",2023-06-29 00:33:07
2024-08-27,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",7,0,"""deactivated""","""e697b430-cbc9-41e3-b0d0-dc02b5…","""d0c82be0-85c3-410a-ae25-413e52…","""2020-12-19 15:03:47""",2023-06-29 00:33:07
2024-08-20,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",7,0,"""deactivated""","""e697b430-cbc9-41e3-b0d0-dc02b5…","""d0c82be0-85c3-410a-ae25-413e52…","""2020-12-19 15:03:47""",2023-06-29 00:33:07
2024-08-20,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",7,0,"""deactivated""","""e697b430-cbc9-41e3-b0d0-dc02b5…","""d0c82be0-85c3-410a-ae25-413e52…","""2020-12-19 15:03:47""",2023-06-29 00:33:07
2024-08-19,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",7,0,"""deactivated""","""e697b430-cbc9-41e3-b0d0-dc02b5…","""d0c82be0-85c3-410a-ae25-413e52…","""2020-12-19 15:03:47""",2023-06-29 00:33:07
…,…,…,…,…,…,…,…,…,…
2023-08-12,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",7,1,"""deactivated""","""e697b430-cbc9-41e3-b0d0-dc02b5…","""d0c82be0-85c3-410a-ae25-413e52…","""2020-12-19 15:03:47""",2023-06-29 00:33:07
2023-08-01,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",7,1,"""deactivated""","""e697b430-cbc9-41e3-b0d0-dc02b5…","""d0c82be0-85c3-410a-ae25-413e52…","""2020-12-19 15:03:47""",2023-06-29 00:33:07
2023-07-31,"""""","""Казань""",7,0,"""deactivated""","""e697b430-cbc9-41e3-b0d0-dc02b5…","""d0c82be0-85c3-410a-ae25-413e52…","""2020-12-19 15:03:47""",2023-06-29 00:33:07


### delete a part


In [22]:
query_text = """--sql
    ALTER TABLE db1.total_active_users_per_day_full_table DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [6]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_full_table_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [7]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_full_table
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.total_active_users_per_day_full_table_mv
"""

ch.query_run(query_text)